# **Processamento de Linguagem Natural [2024-Q2]**
Prof. Alexandre Donizeti Alves

### **PROJETO PRÁTICO** [LangChain + Grandes Modelos de Linguagem + API]


O **PROJETO PRÁTICO** deve ser feitO utilizando o **Google Colab** com uma conta sua vinculada ao Gmail. O link do seu notebook, armazenado no Google Drive, o link de um repositório no GitHub e o link de um vídeo no YouTube do projeto em execução detalhando os principais resultados da atividade, devem ser enviados usando o seguinte formulário:

> Omitido, pois projeto possui repositório no GitHub


**IMPORTANTE**: A submissão deve ser feita até o dia **08/09 (domingo)** APENAS POR UM INTEGRANTE DA EQUIPE, até às 23h59. Por favor, lembre-se de dar permissão de ACESSO IRRESTRITO para o professor da disciplina de PLN.

### **EQUIPE**

---

**Integrante 01:**

Beatriz Sofientini Ribeiro - 11202020433

**Integrante 02:**

Felipe Fernandes Gomes da Silva Costa - 112020202223

**Integrante 03:**

Bruno Francisco Rodrigues Mafra - 11201811147

### **GRANDE MODELO DE LINGUAGEM (*Large Language Model - LLM*)**

---

Cada equipe deve selecionar um Grande Modelo de Linguagem (*Large Language Model - LMM*). Cada modelo pode ser escolhido por até 5 equipes.



Por favor, informe os dados do LLM selecionada:

>


**LLM**: GPT4o

>

**Link para a documentação oficial**: https://platform.openai.com/docs/api-reference



### **API**
---

Por favor, informe os dados da API selecionada:

**API**: API de Receitas

**Site oficial**: https://www.themealdb.com/

**Link para a documentação oficial**: https://www.themealdb.com/api.php






### **DESCRIÇÃO**
---

Implementar um `notebook` no `Google Colab` que faça uso do framework **`LangChain`** (obrigatório) e de um **LLM** aplicando, no mínimo, DUAS técnicas de PLN. As técnicas podem ser aplicada em qualquer córpus obtido a partir de uma **API**.

O **LLM** e a **API** selecionados devem ser informados na seguinte planilha:

> Omitido, pois projeto possui repositório no GitHub
>
As seguintes técnicas de PLN podem ser usadas:

*   Correção Gramatical
*   Classificação de Textos
*   Análise de Sentimentos
*   Detecção de Emoções
*   Extração de Palavras-chave
*   Tradução de Textos
*   Sumarização de Textos
*   Similaridade de Textos
*   Reconhecimento de Entidades Nomeadas
*   Sistemas de Perguntas e Respostas
>

**IMPORTANTE:** É obrigatório usar o e-mail da UFABC.


### **CRITÉRIOS DE AVALIAÇÃO**
---


Serão considerados como critérios de avaliação os seguintes pontos:

* Uso do framework **`LangChain`**.

* Escolha e uso de um **LLM**.

* Escolha e uso de uma **API**.

* Vídeo no YouTube (5 a 10 minutos).

* Criatividade no uso do framework **`LangChain`** em conjunto com o **LLM** e a **API**.




**IMPORTANTE**: todo o código do notebook deve ser executado. Código sem execução não será considerado.

### **IMPLEMENTAÇÃO**
---

### Projeto

O objetivo do projeto é criar um sistema que busca uma receita aleatória na API [The Meal DB](https://www.themealdb.com/), que retorna uma receita descrita em inglês.

Em sequência usando o modelo do GPT-4o implementado com LangChain, deve realizar a tradução dessa receita para português e gerar as seguintes imagens:  uma imagem capa da receita; os ingredientes que são usados; uma imagem do processo de execução da receita. Além disso deve ser gerado informações nutricionais da receita.

Por fim, o texto traduzido e as imagens geradas são publicadas em um [canal de Telegram](https://t.me/+hMmCAxXSZigyYzdh) destinado a compartilhar receitas.

### GitHub

O presente código colab pode ser encontrado no repositório: https://github.com/Mewbi/pln

### Dependências

A primeira etapa é baixar todas as dependências que serão utilizadas no projeto

In [16]:
!pip install -qU langchain langchain_core langchain_openai langchain_community
!pip install -qU python-telegram-bot

In [17]:
#@title Definindo a chave da API da OpenAI, Telegram Token, Channel ID e os modelos que serão utilizados

import getpass
import os

# MODEL_GPT = "gpt-3.5-turbo"
# MODEL_IMAGE = "dall-e-2"

MODEL_GPT = "gpt-4o"
MODEL_IMAGE = "dall-e-3"

print("Será usado o modelo GPT: {}".format(MODEL_GPT))
print("Será usado o modelo de geração de imagem: {}".format(MODEL_IMAGE))

os.environ["OPENAI_API_KEY"] = getpass.getpass('API Key OpenAI: ')
os.environ["TELEGRAM_BOT"] = getpass.getpass('Telegram Bot Token: ')
os.environ["TELEGRAM_CHANNEL"] = getpass.getpass('Telegram Channel ID: ')

Será usado o modelo GPT: gpt-4o
Será usado o modelo de geração de imagem: dall-e-3
API Key OpenAI: ··········
Telegram Bot Token: ··········
Telegram Channel ID: ··········


Em sequência é criado as funções necessárias para buscar uma receita aleatória e formata-la em formato JSON e em formato de texto para tradução.

In [18]:
#@title Definindo funções para buscar e formatar receitas

import requests
import json

# Função para obter uma receita aleatória
def get_random_recipe(debug = False):
    url = "https://www.themealdb.com/api/json/v1/1/random.php"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if data and "meals" in data and data["meals"]:
            return format_response(data["meals"][0], debug)
    return None, None

# Função para processar e exibir os dados da receita
def format_response(recipe, debug):
    if not recipe:
        print("Não foi possível obter a receita.")
        return None, None

    # Separar as informações da receita
    recipe_id = recipe["idMeal"]
    recipe_name = recipe["strMeal"]
    category = recipe["strCategory"]
    area = recipe["strArea"]
    instructions = recipe["strInstructions"]
    ingredients = []

    # Iterar sobre os ingredientes e suas medidas
    for i in range(1, 21):
        ingredient = recipe.get(f"strIngredient{i}")
        measure = recipe.get(f"strMeasure{i}")
        if ingredient and ingredient.strip():
            ingredients.append(f"{measure} {ingredient}")

    # Montar a URL da receita
    recipe_url = f"https://www.themealdb.com/api/json/v1/1/lookup.php?i={recipe_id}"

    response = {
        "id": recipe_id,
        "name": recipe_name,
        "category": category,
        "area": area,
        "ingredients": ingredients,
        "instructions": instructions,
        "url": recipe_url
    }

    response_text = ""
    response_text += f"Nome da Receita: {recipe_name}"
    response_text += f"\nCategoria: {category}"
    response_text += f"\nÁrea: {area}"
    response_text += "\n\nIngredientes:"
    for ingredient in ingredients:
        response_text += f"\n - {ingredient}"
    response_text += "\n\nInstruções: "
    response_text += instructions
    # Exibir as informações da receita
    if debug:
      print(response_text)

    return response, response_text

print("Funções de busca e formatação de receitas para The Meal DB definidas")

Funções de busca e formatação de receitas para The Meal DB definidas


É necessário criar o modelo que Langchain que será responsável para realizar as tarefas de tradução e geração de imagens para as receitas.

In [19]:
#@title Modelos para tradução e geração de imagem

from langchain_openai import ChatOpenAI, OpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains import LLMChain
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper

model = ChatOpenAI(model = MODEL_GPT)

# Criando chain para tradução
parser = StrOutputParser()

template_translation = "Translate the following recipe to {language}"
template_translation_prompt = ChatPromptTemplate.from_messages(
    [("system", template_translation), ("user", "{text}")]
)

chain_translation = template_translation_prompt | model | parser
print("Definido Chain de tradução")

# Criando chain para geração de informações nutricionais
template_nutritional = "Based in the following recipe, give me only the nutritional information of this recipe. Answer in {language}"
template_nutritional_prompt = ChatPromptTemplate.from_messages(
    [("system", template_nutritional), ("user", "{text}")]
)

chain_nutritional = template_nutritional_prompt | model | parser
print("Definido Chain de informações nutricionais")

# Criando modelo de geração de imagem
image_gen = DallEAPIWrapper(
      model = MODEL_IMAGE,
      n = 1,
      quality = "standard",
      size = "1024x1024"
)

llm = OpenAI(temperature=0.9)

# Criando chain para geração de imagem para capa da receita
prompt = PromptTemplate(
    input_variables=["image_desc"],
    template="Generate a image to use as a thumbnail for the recipe {recipe_name}",
)
chain_thumbnail = LLMChain(llm=llm, prompt=prompt)

print("Definido modelo de geração de imagem para capa da receita")

# Criando chain para geração de imagem dos ingredientes da receita
prompt2 = PromptTemplate(
    input_variables=["image_desc"],
    template="Generate a image showing the following ingredients: {ingredients}",
)
chain_ingredients = LLMChain(llm=llm, prompt=prompt2)

print("Definido modelo de geração de imagem para ingredientes da receita")

# Criando chain para geração de imagem das intruções da receita
prompt3 = PromptTemplate(
    input_variables=["image_desc"],
    template="Generate a image with a person doing the following recipe instructions: {instructions}",
)
chain_instructions = LLMChain(llm=llm, prompt=prompt3)

print("Definido modelo de geração de imagem para instruções da receita")


Definido Chain de tradução
Definido Chain de informações nutricionais
Definido modelo de geração de imagem para capa da receita
Definido modelo de geração de imagem para ingredientes da receita
Definido modelo de geração de imagem para instruções da receita


In [20]:
#@title Bot para enviar mensagens e imagens para canal do telegram
import telegram
from telegram import InputMediaPhoto

# Telegram bot token
bot_token = os.environ["TELEGRAM_BOT"]

channel_id = os.environ["TELEGRAM_CHANNEL"]

bot = telegram.Bot(token = bot_token)
bot_name = await bot.getMyName()
print("Conectado ao bot: {}".format(bot_name.name))

Conectado ao bot: Receitas Hummm


Por fim, a última etapa é executar toda a pipeline:
- Buscar uma receita aleatória
- Traduzir toda as suas instruções
- Gerar informações nutricionais da receita
- Gerar uma imagem para a capa
- Gerar uma imagem para os ingredientes
- Gerar uma imagem para as instruções
- Enviar todos os dados para o [telegram](https://t.me/+hMmCAxXSZigyYzdh)

In [21]:
# Obter uma receita aleatória
recipe, recipe_text = get_random_recipe()
print("Buscando receita aleatória...")
if recipe is None:
  print("Erro na busca de uma receita aleatória")
  exit()
print("Receita obtida: {}".format(recipe["name"]))

Buscando receita aleatória...
Receita obtida: Canadian Butter Tarts


In [22]:
print("\nTraduzindo receita...")
recipe_translation = chain_translation.invoke({
    "language": "portuguese",
    "text": recipe_text,
})

print("\nGerando informações nutricionais da receita...")
recipe_nutritional = chain_nutritional.invoke({
    "language": "portuguese",
    "text": recipe_text,
})

print("\nGerando imagem de capa...")
image_thumbnail_url = image_gen.run(chain_thumbnail.run({
    "recipe_name": recipe["name"],
}))

print("\nGerando imagem dos ingredientes...")
# O limite do prompt são 1000 caracteres no total
# O prompt inicial tem cerca de 60 caracteres
ingredients = ""
for ing in recipe["ingredients"]:
  ing = "\n{}".format(ing)
  if len(ingredients + ing) > 1000 - 60:
    break
  ingredients += ing

image_ingredients_url = image_gen.run(chain_ingredients.run({
    "ingredients": ingredients,
}))

# O limite do prompt são 1000 caracteres no total
# O prompt inicial tem cerca de 80 caracteres
instructions = recipe["instructions"][:(1000-80)]
print("\nGerando imagem das instruções...")
image_instructions_url = image_gen.run(chain_instructions.run({
    "instructions": instructions,
}))


Traduzindo receita...

Gerando informações nutricionais da receita...

Gerando imagem de capa...

Gerando imagem dos ingredientes...

Gerando imagem das instruções...


In [23]:
print("\nEnviando para o telegram...")
text = "Receita do dia: {}".format(recipe_translation.splitlines()[0].split(':')[1])

images = [image_thumbnail_url, image_ingredients_url, image_instructions_url]
local_images = False

media_group = []
for idx, image in enumerate(images):
    if not local_images:
        media_group.append(InputMediaPhoto(
            image,
           caption = text if idx == 0 else '')
        )
        continue
    media_group.append(InputMediaPhoto(
        open(image, 'rb'),
        caption = text if idx == 0 else '')
    )

print("\nEnviando imagens para o telegram...")
await bot.send_media_group(chat_id = channel_id, media = media_group)
print("\nEnviando instruções da receita para o telegram...")
_ = await bot.send_message(chat_id = channel_id, text = recipe_translation)
print("\nEnviando informações nutricionais para o telegram...")
_ = await bot.send_message(chat_id = channel_id, text = "Informações Nutricionais\n\n" + recipe_nutritional)
print("\nTodas as informações foram enviadas")


Enviando para o telegram...

Enviando imagens para o telegram...

Enviando instruções da receita para o telegram...

Enviando informações nutricionais para o telegram...

Todas as informações foram enviadas
